In [3]:
from dhanhq import dhanhq, marketfeed
import talib
import numpy as np
import pandas as pd
import time
import datetime
import os
import logging

In [4]:
with open("token.txt", "r") as file:
    # Read the entire contents of the file
    token = file.read()
client_id = "1102249582"
access_token = token
dhan = dhanhq(client_id,access_token)

masterlist = pd.read_csv('https://images.dhan.co/api-data/api-scrip-master.csv', nrows=1)
masterlist = pd.read_csv('https://images.dhan.co/api-data/api-scrip-master.csv', usecols=masterlist.columns, low_memory=False)


logging.basicConfig(filename="logfile.txt", 
                    level=logging.INFO, 
                    format="%(asctime)s - %(levelname)s - %(message)s")


# Common function to place orders
def place_order(security_id, transaction_type, quantity):
    """
    Place an order using dhan API
    
    Args:
        security_id: Security ID to trade
        transaction_type: dhan.BUY or dhan.SELL
        quantity: Quantity to trade
    
    Returns:
        dict: Order response from API or None if failed
    """
    try:
        print("Inside Place Order", security_id, transaction_type, quantity)
        response = dhan.place_order(
            security_id=security_id,
            exchange_segment=dhan.NSE_FNO,
            transaction_type=transaction_type,
            quantity=quantity,
            order_type=dhan.MARKET,
            product_type=dhan.INTRA,
            price=0
        )
        if response and response.get('status') == 'success':
            logging.info(f"Order placed successfully: {response}")
            return response
        else:
            logging.error(f"Order placement failed: {response}")
            return None
    except Exception as e:
        logging.error(f"Error placing order: {e}")
        print(f"❌ Order placement error: {e}")
        return None


In [10]:
NIFTY_PREMIMUM = 25900

filtered_ce_masterlist = masterlist[
    (masterlist.SEM_INSTRUMENT_NAME == 'OPTIDX') &
    (masterlist.SEM_TRADING_SYMBOL.str.startswith('NIFTY')) &
    (masterlist.SEM_EXPIRY_DATE == '2025-11-04 14:30:00') &
    (masterlist.SEM_STRIKE_PRICE == NIFTY_PREMIMUM) &
    (masterlist.SEM_OPTION_TYPE == 'CE') 
]
filtered_ce_masterlist

,SEM_EXM_EXCH_ID,SEM_SEGMENT,SEM_SMST_SECURITY_ID,SEM_INSTRUMENT_NAME,SEM_EXPIRY_CODE,SEM_TRADING_SYMBOL,SEM_LOT_UNITS,SEM_CUSTOM_SYMBOL,SEM_EXPIRY_DATE,SEM_STRIKE_PRICE,SEM_OPTION_TYPE,SEM_TICK_SIZE,SEM_EXPIRY_FLAG,SEM_EXCH_INSTRUMENT_TYPE,SEM_SERIES,SM_SYMBOL_NAME
151272,NSE,D,47676,OPTIDX,0,NIFTY-Nov2025-25900-CE,75.0,NIFTY 04 NOV 25900 CALL,2025-11-04 14:30:00,25900.0,CE,5.0,W,OP,NaN,NaN


In [11]:
filtered_pe_masterlist = masterlist[
    (masterlist.SEM_INSTRUMENT_NAME == 'OPTIDX') &
    (masterlist.SEM_TRADING_SYMBOL.str.startswith('NIFTY')) &
    (masterlist.SEM_EXPIRY_DATE == '2025-11-04 14:30:00') &
    (masterlist.SEM_STRIKE_PRICE == NIFTY_PREMIMUM) &
    (masterlist.SEM_OPTION_TYPE == 'PE')
]

filtered_pe_masterlist

,SEM_EXM_EXCH_ID,SEM_SEGMENT,SEM_SMST_SECURITY_ID,SEM_INSTRUMENT_NAME,SEM_EXPIRY_CODE,SEM_TRADING_SYMBOL,SEM_LOT_UNITS,SEM_CUSTOM_SYMBOL,SEM_EXPIRY_DATE,SEM_STRIKE_PRICE,SEM_OPTION_TYPE,SEM_TICK_SIZE,SEM_EXPIRY_FLAG,SEM_EXCH_INSTRUMENT_TYPE,SEM_SERIES,SM_SYMBOL_NAME
151273,NSE,D,47677,OPTIDX,0,NIFTY-Nov2025-25900-PE,75.0,NIFTY 04 NOV 25900 PUT,2025-11-04 14:30:00,25900.0,PE,5.0,W,OP,NaN,NaN


In [12]:

SECURITY_ID = 47677

IS_ACTIVE = False
IS_OPEN_POSITION = False
IS_FIXED_SL = False
quantity = 150
buy_price = 0     
stop_loss = 0

current_time = datetime.datetime.now()
current_second = current_time.second
time_str = current_time.strftime("%H:%M:%S")

current_date = datetime.datetime.now().strftime("%Y-%m-%d")

while IS_ACTIVE is False:
        current_second = datetime.datetime.now().second
        time.sleep(1)
        print("waiting to activate", current_second)
        if current_second == 0:
            IS_ACTIVE = True

while IS_ACTIVE is True and IS_OPEN_POSITION is False:
      current_second = datetime.datetime.now().second
      if current_second == 0:
            break
      print('Active',current_second)
      time.sleep(0.15)
      intraday = dhan.intraday_minute_data(str(SECURITY_ID), dhan.NSE_FNO, "OPTIDX", current_date, current_date, 1)

      print("intraday", intraday)
      intraday_df = pd.DataFrame(intraday['data']).tail(2)
      temp_list = []

    

      previous_closed_candle = intraday_df.iloc[-2]
      current_running_candle = intraday_df.iloc[-1]
      
      
     
      # if current_running_candle['close'] > previous_closed_candle['high']:
      #       print('Buy')
      #       length_of_previous_candle = round(previous_closed_candle['high'] - previous_closed_candle['low'])
      #       risk                      = 1000
      #       quantity                  = round(round(risk / length_of_previous_candle) / 75) * 75
      #       print('quantity',quantity)
            
      #       if quantity > 200:
      #             quantity = 200

      #       buy_price = int(current_running_candle['close'])
      #       stop_loss = int(previous_closed_candle['low'])

      #       place_order(SECURITY_ID, dhan.BUY, quantity)

      #       logging.info(f"BUY at {buy_price}")

      #       IS_OPEN_POSITION = True
      #       break;
      
      place_order(SECURITY_ID, dhan.BUY, quantity)

      buy_price = float(current_running_candle['close'])
      if IS_FIXED_SL is True: 
            stop_loss = buy_price - 2  #float(previous_closed_candle['low'])
      else:
            stop_loss = float(previous_closed_candle['low'])

      

      logging.info(f"BUY at {buy_price}")
      print(f"BUY at {buy_price}")
      print(f"STOP LOSS at {stop_loss}") 

      IS_OPEN_POSITION = True
      break




waiting to activate 46
waiting to activate 47
waiting to activate 48
waiting to activate 49
waiting to activate 50
waiting to activate 51
waiting to activate 52
waiting to activate 53
waiting to activate 54
waiting to activate 55
waiting to activate 56
waiting to activate 57
waiting to activate 58
waiting to activate 59
waiting to activate 0
Active 1
intraday {'status': 'success', 'remarks': '', 'data': {'open': [58.8, 79.1, 80.5, 79.6, 75.3, 78.15, 76.75, 80.5, 80.05, 77.05, 79.85, 79.9, 79.9, 79.4, 78.9, 79.55, 85.1, 89.2, 95.5, 99.1, 102.75, 97.7, 98.4, 105.35], 'high': [79.95, 81.7, 81.0, 80.05, 76.95, 79.75, 79.7, 87.9, 80.25, 79.75, 82.05, 81.3, 79.9, 81.05, 80.85, 85.2, 90.0, 96.0, 99.8, 103.25, 103.65, 98.3, 105.85, 105.35], 'low': [58.8, 77.75, 78.6, 74.5, 73.5, 76.15, 75.85, 79.8, 76.95, 77.0, 79.25, 78.2, 74.75, 78.1, 77.9, 79.55, 84.45, 87.7, 93.4, 98.5, 97.6, 95.35, 98.25, 104.6], 'close': [76.15, 79.45, 79.15, 74.5, 76.95, 77.3, 79.7, 80.55, 77.55, 79.3, 80.75, 79.3, 78.6

In [13]:

COST_TO_COST_SL = False
TRAILING_SL = False

print(f"Starting position monitoring...")
print(f"Position Status: IS_OPEN_POSITION = {IS_OPEN_POSITION}, Quantity = {quantity}")
print(f"Stop Loss Settings: COST_TO_COST_SL = {COST_TO_COST_SL}, TRAILING_SL = {TRAILING_SL}")
print(f"Buy Price: {buy_price}, Stop Loss: {stop_loss}")
print("-" * 50)

while IS_OPEN_POSITION is True and quantity > 0:
      time.sleep(0.10)
      
      try:
          # Fetch fresh data for real-time monitoring
          intraday = dhan.intraday_minute_data(str(SECURITY_ID), dhan.NSE_FNO, "OPTIDX", current_date, current_date, 1)
          
          # Check if data is available and valid
          if intraday['status'] != 'success' or not intraday.get('data'):
              print("❌ No data available for monitoring, retrying...")
              continue
              
          # Safely create DataFrame with error handling
          data = intraday['data']
          
          # Check if data is a dictionary with lists (expected format)
          if isinstance(data, dict) and all(isinstance(v, list) for v in data.values()):
              intraday_df = pd.DataFrame(data).tail(2)
          else:
              print(f"❌ Unexpected data format: {type(data)}")
              continue
              
          if len(intraday_df) < 2:
              print("❌ Insufficient data for monitoring, retrying...")
              continue

          previous_closed_candle = intraday_df.iloc[-2]
          current_running_candle = intraday_df.iloc[-1]
          
          current_price = current_running_candle['close']
          previous_low = previous_closed_candle['low']
          
          # Calculate P&L
          price_diff = current_price - buy_price
          pnl = price_diff * quantity
          pnl_percentage = (price_diff / buy_price) * 100
          
          # Add timestamp for monitoring
          current_time = datetime.datetime.now().strftime("%H:%M:%S")
          
          # Format P&L display
          if pnl >= 0:
              pnl_display = f"📈 +₹{pnl:.2f} (+{pnl_percentage:.2f}%)"
          else:
              pnl_display = f"📉 ₹{pnl:.2f} ({pnl_percentage:.2f}%)"
          
          # Show appropriate stop loss monitoring based on settings
          if COST_TO_COST_SL is True and TRAILING_SL is False:
              # Cost-to-Cost SL only
              ctc_sl_level = buy_price + 1
              print(f"⏰ {current_time} | 📊 Current: {current_price:.2f} | 🎯 SL: {stop_loss:.2f} | 💰💰 CTC SL: {ctc_sl_level:.2f} | 💰 Buy: {buy_price:.2f} | {pnl_display}")
          elif TRAILING_SL is True and COST_TO_COST_SL is False:
              # Trailing SL only
              print(f"⏰ {current_time} | 📊 Current: {current_price:.2f} | 🎯 SL: {stop_loss:.2f} | 🔄 Trail SL: {previous_low:.2f} | 💰 Buy: {buy_price:.2f} | {pnl_display}")
          elif COST_TO_COST_SL is True and TRAILING_SL is True:
              # Both CTC and Trailing SL
              ctc_sl_level = buy_price + 1
              print(f"⏰ {current_time} | 📊 Current: {current_price:.2f} | 🎯 SL: {stop_loss:.2f} | 💰💰 CTC SL: {ctc_sl_level:.2f} | 🔄 Trail SL: {previous_low:.2f} | 💰 Buy: {buy_price:.2f} | {pnl_display}")
          else:
              # Regular SL only
              print(f"⏰ {current_time} | 📊 Current: {current_price:.2f} | 🎯 SL: {stop_loss:.2f} | 💰 Buy: {buy_price:.2f} | {pnl_display}")

          # Stop loss
          if current_price < stop_loss and COST_TO_COST_SL is False and TRAILING_SL is False:
                print("🔴 STOP LOSS TRIGGERED!")
                print(f"Current Price ({current_price:.2f}) < Stop Loss ({stop_loss:.2f})")
                place_order(SECURITY_ID, dhan.SELL, quantity)
                print(f"✅ SL hit at {current_price:.2f}")
                print(f"💸 Final P&L: ₹{pnl:.2f} ({pnl_percentage:.2f}%)")
                IS_OPEN_POSITION = False
                break

          # Trailing SL
          if current_price < previous_low and COST_TO_COST_SL is False and TRAILING_SL is True:
                print("🔴 TRAILING STOP LOSS TRIGGERED!")
                print(f"Current Price ({current_price:.2f}) < Previous Low ({previous_low:.2f})")
                place_order(SECURITY_ID, dhan.SELL, quantity)
                print(f"✅ Trailing SL hit at {current_price:.2f}")
                print(f"💸 Final P&L: ₹{pnl:.2f} ({pnl_percentage:.2f}%)")
                IS_OPEN_POSITION = False
                break

          # Shift sl to Cost to Cost
          if current_price < buy_price + 1 and COST_TO_COST_SL is True and TRAILING_SL is False:
                print("🔴 COST TO COST STOP LOSS TRIGGERED!")
                print(f"Current Price ({current_price:.2f}) < Buy Price + 1 ({buy_price + 1:.2f})")
                place_order(SECURITY_ID, dhan.SELL, quantity)
                print(f"✅ CTC SL hit at {current_price:.2f}")
                print(f"💸 Final P&L: ₹{pnl:.2f} ({pnl_percentage:.2f}%)")
                IS_OPEN_POSITION = False
                break
                
      except Exception as e:
          print(f"❌ Error monitoring position: {e}")
          time.sleep(0.1)  # Brief pause on error
          continue

print("🏁 Position monitoring ended!")

            

Starting position monitoring...
Position Status: IS_OPEN_POSITION = True, Quantity = 150
Stop Loss Settings: COST_TO_COST_SL = False, TRAILING_SL = False
Buy Price: 104.6, Stop Loss: 98.25
--------------------------------------------------
⏰ 09:38:13 | 📊 Current: 100.95 | 🎯 SL: 98.25 | 💰 Buy: 104.60 | 📉 ₹-547.50 (-3.49%)
⏰ 09:38:13 | 📊 Current: 100.95 | 🎯 SL: 98.25 | 💰 Buy: 104.60 | 📉 ₹-547.50 (-3.49%)
⏰ 09:38:13 | 📊 Current: 101.30 | 🎯 SL: 98.25 | 💰 Buy: 104.60 | 📉 ₹-495.00 (-3.15%)
⏰ 09:38:13 | 📊 Current: 101.30 | 🎯 SL: 98.25 | 💰 Buy: 104.60 | 📉 ₹-495.00 (-3.15%)
⏰ 09:38:13 | 📊 Current: 101.30 | 🎯 SL: 98.25 | 💰 Buy: 104.60 | 📉 ₹-495.00 (-3.15%)
❌ No data available for monitoring, retrying...
❌ No data available for monitoring, retrying...
❌ No data available for monitoring, retrying...
⏰ 09:38:14 | 📊 Current: 101.30 | 🎯 SL: 98.25 | 💰 Buy: 104.60 | 📉 ₹-495.00 (-3.15%)
⏰ 09:38:14 | 📊 Current: 101.40 | 🎯 SL: 98.25 | 💰 Buy: 104.60 | 📉 ₹-480.00 (-3.06%)
⏰ 09:38:14 | 📊 Current: 101.40 | 🎯 

KeyboardInterrupt: 